<a href="https://colab.research.google.com/github/yudinatatiana/Comp-ling_computational_linguistics/blob/main/Ngrams.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Языковое моделирование

Языковое моделирование заключается в приписывании вероятности последовательности слов. Сейчас языковые модели используются практически во всех nlp задачах. Всякие Берты и Элмо - языковые модели. 

Это достаточно сложная тема, поэтому будем разбирать постепенно. Сегодня разберём самые основы. Научимся приписывать вероятность последовательности слов и попробуем генерировать текст.

Возьмем два разных корпуса: новостной и сообщения с 2ch.

In [1]:
import os
os.listdir()

['.config', 'lenta.txt', '2ch_corpus.txt', 'sample_data']

In [3]:
# !!! двач не самое приятное место, большое количество текстов в этом корпусе токсичные
dvach = open('2ch_corpus.txt').read()[:300000]
# !!! двач не самое приятное место, большое количество текстов в этом корпусе токсичные

news = open('lenta.txt').read()[:300000]

По длине оно сопоставимы.

In [3]:
print("Длина 1 -", len(dvach))
print("Длина 2 -", len(news))

Длина 1 - 400000
Длина 2 - 400000


Напишем простую функцию для нормализации. 

In [3]:
!pip install razdel

In [4]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np

def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text


Сравним тексты по токенам

In [5]:
norm_dvach = normalize(dvach)
norm_news = normalize(news)

In [7]:
print("Длина корпуса токсичных постов в токенах -", len(norm_dvach))
print("Длина корпуса новостных текстов в токенах - ", len(norm_news))

Длина корпуса токсичных постов в токенах - 63294
Длина корпуса новостных текстов в токенах -  51538


И по уникальным токенам

In [8]:
print("Уникальных токенов в корпусе токсичных постов -", len(set(norm_dvach)))
print("Уникальный токенов в корпусе новостных текстов - ", len(set(norm_news)))

Уникальных токенов в корпусе токсичных постов - 16693
Уникальный токенов в корпусе новостных текстов -  15345


Посчитаем, сколько раз встречаются слова и выведем самые частотные.

In [6]:
from collections import Counter

In [10]:
vocab_dvach = Counter(norm_dvach)
vocab_news = Counter(norm_news)


In [11]:
vocab_dvach.most_common(10)

[('и', 1870),
 ('не', 1638),
 ('в', 1615),
 ('на', 1041),
 ('что', 949),
 ('а', 751),
 ('я', 683),
 ('это', 643),
 ('с', 613),
 ('как', 577)]

In [12]:
vocab_news.most_common(10)

[('в', 2447),
 ('и', 1209),
 ('на', 883),
 ('по', 702),
 ('с', 566),
 ('что', 560),
 ('не', 422),
 ('как', 322),
 ('из', 296),
 ('о', 268)]

Сравнивать употребимость конкретных слов в разных текстах в абсолютных числах неудобно. Нормализуем счётчики на размеры текстов. Так у нас получается вероятность слова.

In [13]:
probas_dvach = Counter({word:c/len(norm_dvach) for word, c in vocab_dvach.items()})
probas_dvach.most_common(20)

[('и', 0.02954466458116093),
 ('не', 0.025879230258792303),
 ('в', 0.025515846683729894),
 ('на', 0.016447056593041996),
 ('что', 0.014993522292792365),
 ('а', 0.011865263690081208),
 ('я', 0.010790912250766265),
 ('это', 0.010158940815875123),
 ('с', 0.009684962239706765),
 ('как', 0.009116187948304736),
 ('ты', 0.008958195089581952),
 ('но', 0.006509305779378772),
 ('у', 0.006051126489082693),
 ('то', 0.005671943628148008),
 ('так', 0.005419155054191551),
 ('если', 0.004992574335640029),
 ('все', 0.004866180048661801),
 ('же', 0.0046449900464499),
 ('он', 0.004486997187727115),
 ('по', 0.004107814326792429)]

In [14]:
probas_news = Counter({word:c/len(norm_news) for word, c in vocab_news.items()})
probas_news.most_common(20)

[('в', 0.04747952966742986),
 ('и', 0.023458419030618186),
 ('на', 0.017132989250650005),
 ('по', 0.013621017501649268),
 ('с', 0.01098218790019015),
 ('что', 0.010865768947184601),
 ('не', 0.008188133028056968),
 ('как', 0.006247817144631146),
 ('из', 0.005743335014940432),
 ('о', 0.005200046567581202),
 ('к', 0.0041328728316970004),
 ('россии', 0.003977647561022935),
 ('за', 0.003395552795995188),
 ('также', 0.0030851022546470566),
 ('для', 0.00304629593697854),
 ('от', 0.003026892778144282),
 ('его', 0.0030074896193100237),
 ('сегодня', 0.0029880864604757656),
 ('он', 0.002891070666304474),
 ('а', 0.002755248554464667)]

Эти вероятности уже можно использовать, чтобы ответить на вопрос - это предложение больше подходит для новостей или для анонимного форума?

In [15]:
phrase = 'Технические возможности устаревшего российского судна не позволили разгрузить его у терминала'

prob = Counter({'news':0, 'dvach':0})

for word in normalize(phrase):
    prob['dvach'] += probas_dvach.get(word, 0)
    prob['news'] += probas_news.get(word, 0)



In [16]:
prob.most_common()

[('dvach', 0.03461623534616236), ('news', 0.013640420660483528)]

In [17]:
phrase = 'как вы смотрите эту залупу, серьезно. в чем прикол ваще это смотреть?'

prob = Counter({'news':0, 'dvach':0})

for word in normalize(phrase):
    prob['dvach'] += probas_dvach.get(word, 0)
    prob['news'] += probas_news.get(word, 0)



In [18]:
prob.most_common()

[('news', 0.05673483643137103), ('dvach', 0.04891458906057446)]

Результаты получаются не очень точные. Возможно это из-за того, что мы считаем слова независимыми друг от друга. А это очевидно не так

По-хорошему вероятность последовательности нужно расчитывать по формуле полной вероятности. Но у нас не очень большие тексты и мы не можем получить вероятности для длинных фраз (их просто может не быть в текстах). Поэтому мы воспользуемся предположением Маркова и будем учитывать только предыдущее слово.

Чтобы расчитать вероятность с таким предположением, нам достаточно найти количество вхождений для каждого биграмма.

In [7]:
from nltk.tokenize import sent_tokenize
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [20]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Для того, чтобы у нас получились честные вероятности и можно было посчитать вероятность первого слова, нам нужно добавить тэг маркирующий начало предложений \< start \>

Дальше мы попробуем сгенерировать текст, используя эти вероятности, и нам нужно будет когда-то остановится. Для этого добавим тэг окончания \< end \>

Ну и поделим все на предложения

In [21]:
sentences_dvach = [['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dvach)]
sentences_news = [['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news)]

In [22]:
unigrams_dvach = Counter()
bigrams_dvach = Counter()

for sentence in sentences_dvach:
    unigrams_dvach.update(sentence)
    bigrams_dvach.update(ngrammer(sentence))


unigrams_news = Counter()
bigrams_news = Counter()

for sentence in sentences_news:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence))


In [23]:
len(unigrams_dvach)

16695

In [24]:
bigrams_news.most_common(10)

[('<start> в', 244),
 ('<start> по', 212),
 ('<start> как', 142),
 ('риа новости', 84),
 ('<start> на', 71),
 ('по словам', 64),
 ('что в', 60),
 ('в москве', 58),
 ('об этом', 57),
 ('<start> об', 53)]

Чтобы посчитать условную вероятность мы можем поделить количество вхождений на количество вхождений первого слова.

In [25]:
phrase = 'Технические возможности устаревшего российского судна не позволили разгрузить его у терминала''Ныть надо меньше и работать больше.'
# phrase = 'как вы смотрите эту залупу, серьезно. в чем прикол ваще это смотреть?'
prob = Counter()
for ngram in ngrammer(['<start>'] + normalize(phrase) + ['<end>']):
    word1, word2 = ngram.split()
    
    if word1 in unigrams_dvach and ngram in bigrams_dvach:
        prob['dvach'] += np.log(bigrams_dvach[ngram]/unigrams_dvach[word1])
    else:
        prob['dvach'] += np.log(0.001)
    
    if word1 in unigrams_news and ngram in bigrams_news:
        prob['news'] += np.log(bigrams_news[ngram]/unigrams_news[word1])
    else:
        prob['news'] += np.log(0.001)

prob['news'] = np.exp(prob['news'])
prob['dvach'] = np.exp(prob['dvach'])

In [26]:
prob.most_common()

[('dvach', 5.941770647653036e-50), ('news', 1.000000000000004e-51)]

Работает получше. Мы воспользовались небольшим хаком - для слов или биграммов, которых не было у нас в словаре, прибавляли низкую вероятность. Исправить это по-нормальному - сложно, придется подробнее разбираться с вероятностями, сглаживаниями и заменой неизвестных слов. Если интрересно - в книге Журафского про это есть.

Проблем с неизвестными словами у нас не будет, если мы будем пытаться сгенерировать новый текст. Давайте попробуем это сделать.

In [27]:
matrix_dvach = np.zeros((len(unigrams_dvach), 
                   len(unigrams_dvach)))
id2word_dvach = list(unigrams_dvach)
word2id_dvach = {word:i for i, word in enumerate(id2word_dvach)}


for ngram in bigrams_dvach:
    word1, word2 = ngram.split()
    matrix_dvach[word2id_dvach[word1]][word2id_dvach[word2]] =  (bigrams_dvach[ngram]/
                                                                     unigrams_dvach[word1])



In [28]:
# создадим матрицу вероятностей перейти из 1 слов в другое
matrix_news = np.zeros((len(unigrams_news), 
                   len(unigrams_news)))

id2word_news = list(unigrams_news)
word2id_news = {word:i for i, word in enumerate(id2word_news)}


# вероятность расчитываем точно также
for ngram in bigrams_news:
    word1, word2 = ngram.split()
    matrix_news[word2id_news[word1]][word2id_news[word2]] =  (bigrams_news[ngram]/
                                                                     unigrams_news[word1])



Для генерации нам понадобится функция np.random.choice , которая выбирает случайный объект из заданных. Ещё в неё можно подать вероятность каждого объекта и она будет доставать по ним (не только максимальный по вероятности)

In [29]:

def generate(matrix, id2word, word2id, n=100, start='<start>'):
    text = []
    current_idx = word2id[start]
    
    for i in range(n):
        
        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx])
        text.append(id2word[chosen])
        
        if id2word[chosen] == '<end>':
            chosen = word2id['<start>']
        current_idx = chosen
    
    return ' '.join(text)

In [30]:
print(generate(matrix_dvach, id2word_dvach, word2id_dvach).replace('<end>', '\n'))

не реализован во наличие прогрессивного веб-фрейворка адекватная замена рельсам на вопрос риторический 
 меня ненавидеть но тут только тогда 
 прочитал ударника в секцию ходил как только правило построения твоим словам еще эпигаиральная фаза когда уже срать 
 через 5 6 всего 56 учеников это точно обозначены все треды с параши 
 скоро волью новую точку соединил с определением круга общения с ношпой принять за тобой 
 освятил на танцы 
 чтоб глаз – это же охуенным 
 потом напишу стихотворение о людях больше 
 1 
 да я сегодня у вас есть такая же в бикини в жизни и


In [31]:
print(generate(matrix_news, id2word_news, word2id_news).replace('<end>', '\n'))

нам кажется что эта секта в изготовлении бомб 
 более дорогие g 4 
 президенты также проведена операция получившая название cbs в правоохранительных органах курска корреспонденту итар-тасс согласно которому он высказал мнение что в течение двух зданий в центре им потребуется ровно год назад строев председатель облизбиркома список претендентов 
 теперь куплю 
 дело против россии без права и законных интересов граждан 
 бой продолжался полчаса 
 диггеры решили уехать в случае как мэра столицы уточнил он 
 здесь уже начала расследование чтобы на помощь частных предприятий связи объяснили свой долг по оценке представителя 
 по подъезду в британскую лабораторию


Попробуйте сделать триграммную модель на основе кода выше.

Подсказки:
    - нужно будет добавить еще один тэг <start>
    - еще одна матрица не нужна, можно по строкам хронить биграмы, а по колонкам униграммы
    - тексты должны быть очень похоже на нормальные (если у вас получается рандомная каша, вы что-то делаете не так)

# Задача 1

In [8]:
sentences_dvach = [['<start>', '<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dvach)]
sentences_news = [['<start>', '<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news)]

In [9]:
unigrams_dvach = Counter()
bigrams_dvach = Counter()
threegrams_dvach = Counter()

for sentence in sentences_dvach:
    unigrams_dvach.update(sentence)
    bigrams_dvach.update(ngrammer(sentence))
    threegrams_dvach.update(ngrammer(sentence, n=3))


unigrams_news = Counter()
bigrams_news = Counter()
threegrams_news = Counter()

for sentence in sentences_news:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence))
    threegrams_news.update(ngrammer(sentence, n=3))

In [10]:
matrix_dvach = np.zeros((len(bigrams_dvach), 
                   len(unigrams_dvach)))
id2word_dvach = list(unigrams_dvach)
word2id_dvach = {word:i for i, word in enumerate(id2word_dvach)}

id2bigrams_dvach = list(bigrams_dvach)
bigram2id_dvach = {bigram:i for i, bigram in enumerate(id2bigrams_dvach)}


for ngram in threegrams_dvach:
    word1, word2, word3 = ngram.split()
    matrix_dvach[bigram2id_dvach[(word1 + ' ' + word2)]][word2id_dvach[word3]] =  (threegrams_dvach[ngram]/
                                                                     bigrams_dvach[(word1 + ' ' + word2)])

In [12]:
# создадим матрицу вероятностей перейти из биграммы в триграмму
matrix_news = np.zeros((len(bigrams_news), 
                   len(unigrams_news)))

id2word_news = list(unigrams_news)
word2id_news = {word:i for i, word in enumerate(id2word_news)}

id2bigrams_news = list(bigrams_news)
bigram2id_news = {bigram:i for i, bigram in enumerate(id2bigrams_news)}


# вероятность расчитываем точно также
for ngram in threegrams_news:
    word1, word2, word3 = ngram.split()
    matrix_news[bigram2id_news[(word1 + ' ' + word2)]][word2id_news[word3]] =  (threegrams_news[ngram]/
                                                                     bigrams_news[(word1 + ' ' + word2)])

In [13]:
def generate(matrix, id2word, word2id, id2bigram, bigram2id, n=100, start='<start> <start>'):
    text = []
    current_idx = bigram2id[start]
    
    for i in range(n):
        
        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx])
        text.append(id2word[chosen])
       
        if id2word[chosen] == '<end>':
            chosen = bigram2id['<start> <start>']
        else:
            chosen = bigram2id[(id2bigram[current_idx].split()[1] + ' ' + id2word[chosen])]
        current_idx = chosen
    
    return ' '.join(text)

In [19]:
print(generate(matrix_news, id2word_news, word2id_news, id2bigrams_news, bigram2id_news).replace('<end>', '\n'))

с тем что исаков больше не возьмем его заявил он 
 в ходеплебисцита 78,5 жителей восточного тимора дили планируют завтра эвакуировать большинство своих сотрудников несмотря на отсутствие у него намерении баллотироватьсяна пост мэра москвы а недовольны его работой лишь 12 жителей столицы россии предпочли бы видеть 5 опрошенных ни на что они также стали жертвами оползня 
 там же находится главный претендент на президентский пост мэр буэнос-айреса фернандо де ла руа 
 глава mabetex не ожидает что против него в суд с иском моральный ущерб руководитель центрального избирательного штаба блока георгием боосом и секретарем координационного совета блока депутатом госдумы олегом


In [18]:
print(generate(matrix_dvach, id2word_dvach, word2id_dvach, id2bigrams_dvach, bigram2id_dvach).replace('<end>', '\n'))

в мангаче 
 за что так происходит обычно 
 какое-то хуевое видео 
 мимо иногда играю в хорошем разрешении со сглаживанием в старые вины 
 ебать чё как в районе и все кто так делают неимоверно кайфуют от того чтобы знать что на работу возьмут хачкель писать я думаю что ты брат надо себя заставлять 
 начинаю мерить темпу картина другая и по еще сотне подобных причин 
 привет всем новоприбывшим 
 а зачем с порнографией боретесь вы че дети мизулиной да что вы смотрели 
 я думаю тоже нужно работать 
 в гугле ориентация больше идёт на молодых и


# Задача 2

In [26]:
import itertools
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
from pymorphy2 import MorphAnalyzer
from collections import Counter, defaultdict
import numpy as np
import re
from string import punctuation
from nltk.corpus import stopwords

stops = set(stopwords.words('russian') + ["это", "весь"])
morph = MorphAnalyzer()

def normalize(text):
    tokens = re.findall('[а-яёa-z0-9]+', text.lower())
    normalized_text = [morph.parse(word)[0].normal_form for word \
                                                            in tokens]
    normalized_text = [word for word in normalized_text if len(word) > 2 and word not in stops]
    
    return normalized_text

def preprocess(text):
    sents = sentenize(text)
    return [normalize(sent.text) for sent in sents]

def ngrammer(tokens, stops, n=2):
    ngrams = []
    tokens = [token for token in tokens if token not in stops]
    for i in range(0,len(tokens)-n+1):
        ngrams.append(tuple(tokens[i:i+n]))
    return ngrams

In [21]:
!pip install pymorphy2

     |████████████████████████████████| 61kB 2.9MB/s 
     |████████████████████████████████| 8.2MB 5.3MB/s 


In [43]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [44]:
news = open('lenta.txt').read()

In [45]:
corpus = preprocess(news)

In [55]:
def scorer(worda_count, wordb_count, bigram_count, len_vocab, min_count, corpus_word_count):
    if bigram_count >= min_count:
        prop_a = worda_count / corpus_word_count
        prop_b = wordb_count / corpus_word_count
        prop_ab = bigram_count / corpus_word_count
        try: 
            return np.log(prop_ab / (prop_a * prop_b))
        except ValueError:
            return -1000
    else:
        return -1000

In [56]:
import gensim

# собираем статистики
ph = gensim.models.Phrases(corpus, scoring=scorer, threshold=0)

# преобразовывать можно и через ph, но так быстрее 
p = gensim.models.phrases.Phraser(ph)

# собираем статистики по уже забиграммленному тексту
ph2 = gensim.models.Phrases(p[corpus], scoring=scorer, threshold=0)
p2 = gensim.models.phrases.Phraser(ph2)

p2[p[corpus[333]]]

['установить',
 'взрыв_произойти',
 'третий',
 'уровнечетвертый',
 'ярус',
 'комплекс',
 'зал',
 'игровой_автомат']

In [60]:
# собираем статистики
ph = gensim.models.Phrases(corpus, scoring=scorer, threshold=4.5)

# преобразовывать можно и через ph, но так быстрее 
p = gensim.models.phrases.Phraser(ph)

# собираем статистики по уже забиграммленному тексту
ph2 = gensim.models.Phrases(p[corpus], scoring=scorer, threshold=4.5)
p2 = gensim.models.phrases.Phraser(ph2)

p2[p[corpus[333]]]

['установить',
 'взрыв',
 'произойти',
 'третий',
 'уровнечетвертый',
 'ярус',
 'комплекс',
 'зал',
 'игровой_автомат']

In [63]:
# собираем статистики
ph = gensim.models.Phrases(corpus, scoring=scorer, threshold=7.5)

# преобразовывать можно и через ph, но так быстрее 
p = gensim.models.phrases.Phraser(ph)

# собираем статистики по уже забиграммленному тексту
ph2 = gensim.models.Phrases(p[corpus], scoring=scorer, threshold=7.5)
p2 = gensim.models.phrases.Phraser(ph2)

p2[p[corpus[333]]]

['установить',
 'взрыв',
 'произойти',
 'третий',
 'уровнечетвертый',
 'ярус',
 'комплекс',
 'зал',
 'игровой',
 'автомат']

При значении threshold = 0 попадали обе биграммы, при постепенном увеличении биграммы начинали пропадать